### 🎯 Effet d'information — entre précision et exactitude

Dans le monde complexe de l’estimation des ressources minières, l’information est reine. Plus vous avez de données de qualité, plus votre image du gisement devient nette — comme une mise au point en photographie. Mais attention : quantité ne rime pas toujours avec qualité.

🔍 Deux notions fondamentales entrent en jeu :

- **🎯 Précision**  
  C’est la capacité à reproduire le même résultat à chaque mesure. Imaginez qu’on analyse plusieurs fois une même carotte de forage : si les résultats sont serrés, même s’ils ne sont pas exacts, la précision est bonne. Cela signifie que **le bruit aléatoire est faible**.

- **📏 Exactitude**  
  C’est la capacité à viser autour de la cible systématiquement. Une méthode d’analyse peut être précise, mais toujours décalée vers le haut ou le bas — c’est un **biais systématique**. On touche la cible… mais à côté du centre. On exact lorsque l'on est proche de la cible.

✅ Lorsque les données sont à la fois précises et exactes, on dit qu’elles sont **justes**. C’est l’idéal, mais aussi un objectif ambitieux en géostatistique. En pratique, les données géologiques sont inévitablement entachées d’erreurs de diverses natures : erreurs de mesure en laboratoire, imprécision des sondages, approximations d’interprétation, ou encore biais introduits par les méthodes d’estimation. Comprendre et quantifier ces incertitudes est essentiel pour une prise de décision responsable.

---

Dans la réalité du terrain, les teneurs ne sont **jamais parfaitement connues** avant extraction. On les estime à partir d’informations comme les forages ou les analyses en laboratoire. Après l’extraction, on peut enfin comparer les **teneurs estimées** avec les **teneurs mesurées** pour détecter les erreurs ou biais.

Les **écarts** entre estimation et réalité peuvent venir :

- de la **variabilité aléatoire** des données (→ manque de précision),
- ou d’un **biais systématique** dans la méthode (→ manque d’exactitude).

---

### 🧪 Explorez par vous-même !

Grâce au graphique interactif ci-dessous, vous pouvez tester vous-même l’effet :

- du **bruit aléatoire** (manque de précision),
- et du **biais systématique** (manque d’exactitude),

sur les cartes de teneur et le nuage de points estimé vs réel.

**💡 Essayez différents scénarios** : ajoutez du bruit, introduisez un biais… et observez l’impact sur les décisions minières !


In [17]:

import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft2, ifft2, fftshift
import ipywidgets as widgets
from ipywidgets import interact

def spherical_covariance_fft(size, range_, sill=1.0):
    extended_size = 2 * size
    x = np.arange(-extended_size//2, extended_size//2)
    X, Y = np.meshgrid(x, x)
    h = np.sqrt(X**2 + Y**2)
    h = np.minimum(h, range_ * size)
    cov = sill * (1 - 1.5 * (h / (range_ * size)) + 0.5 * (h / (range_ * size))**3)
    cov[h > (range_ * size)] = 0
    return fftshift(cov)

def fftma_simulation(size, range_, sill=1.0, seed=0):
    np.random.seed(seed)
    extended_size = 2 * size
    cov_model = spherical_covariance_fft(size, range_, sill)
    cov_fft = fft2(cov_model)
    white_noise = np.random.normal(size=(extended_size, extended_size))
    white_fft = fft2(white_noise)
    z_fft = np.sqrt(np.abs(cov_fft)) * white_fft
    z_ext = np.real(ifft2(z_fft))
    start = extended_size // 4
    end = start + size
    return z_ext[start:end, start:end]

def gaussian_to_lognormal(field):
    return np.exp(field)

def plot_real_vs_estimated_bias(bias_percent=0, noise_std=0.5, cutoff=2):
    size = 200
    seed = 42
    np.random.seed(seed)
    
    field = fftma_simulation(size=size, range_=0.2, sill=1.0, seed=seed)
    real = gaussian_to_lognormal(field)
    
    # Ajout du biais multiplicatif (en %)
    biased_real = real * (1 + bias_percent / 100)
    
    # Bruit multiplicatif log-normal centré pour espérance égale 1
    epsilon = np.random.normal(loc=0, scale=noise_std, size=real.shape)
    
    # Estimation bruitée = biais * bruit multiplicatif centré
    estimated = biased_real + epsilon
    
    lower_clip, upper_clip = 0, 10
    real_clipped = np.clip(real, lower_clip, upper_clip)
    estimated_clipped = np.clip(estimated, lower_clip, upper_clip)
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    im0 = axes[0].imshow(real_clipped, cmap='viridis', vmin=lower_clip, vmax=upper_clip)
    axes[0].set_title("Champ réel")
    axes[0].set_xlabel("X (m)")
    axes[0].set_ylabel("Y (m)")
    fig.colorbar(im0, ax=axes[0], fraction=0.046, pad=0.04)

    im1 = axes[1].imshow(estimated_clipped, cmap='viridis', vmin=lower_clip, vmax=upper_clip)
    axes[1].set_title("Champ estimé")
    axes[1].set_xlabel("X (m)")
    axes[1].set_ylabel("Y (m)")
    fig.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)

    real_flat = real_clipped.flatten()
    estimated_flat = estimated_clipped.flatten()

    mask_red = (real_flat >= cutoff) & (estimated_flat < cutoff)
    mask_blue = (real_flat < cutoff) & (estimated_flat >= cutoff)
    mask_other = ~(mask_red | mask_blue)

    axes[2].scatter(estimated_flat[mask_other], real_flat[mask_other], alpha=0.3, s=10, color="gray", edgecolor="none", label="Correctement classé")
    axes[2].scatter(estimated_flat[mask_blue], real_flat[mask_blue], alpha=0.7, s=15, color="blue", edgecolor="k", linewidth=0.2, label="Stérile traité")
    axes[2].scatter(estimated_flat[mask_red], real_flat[mask_red], alpha=0.7, s=15, color="red", edgecolor="k", linewidth=0.2, label="Minerai ignoré")

    axes[2].plot([lower_clip, upper_clip], [lower_clip, upper_clip], 'k-', linewidth=2, label="Ligne 1:1")

    # Régression linéaire inversée (estimé -> réel)
    A = np.vstack([estimated_flat, np.ones_like(estimated_flat)]).T
    a, b = np.linalg.lstsq(A, real_flat, rcond=None)[0]
    x_fit = np.array([lower_clip, upper_clip])
    y_fit = a * x_fit + b
    axes[2].plot(x_fit, y_fit, 'r--', label=f"Régression linéaire\nréel={a:.2f}*estimé+{b:.2f}")

    axes[2].axhline(cutoff, color='gray', linestyle='--')
    axes[2].axvline(cutoff, color='gray', linestyle='--')

    axes[2].set_xlabel("Teneur estimée (ppm)")
    axes[2].set_ylabel("Teneur réelle (ppm)")
    axes[2].set_title("Teneur réelle vs estimée")
    axes[2].set_xlim(lower_clip, upper_clip)
    axes[2].set_ylim(lower_clip, upper_clip)
    axes[2].set_aspect('equal', adjustable='box')

    Q1 = np.sum((real_flat >= cutoff) & (estimated_flat >= cutoff))
    Q2 = np.sum(mask_red)
    Q3 = np.sum((real_flat < cutoff) & (estimated_flat < cutoff))
    Q4 = np.sum(mask_blue)
    total = len(real_flat)

    axes[2].text(0.2, cutoff - 0.5, f'{Q1/total*100:.1f}%', fontsize=12)
    axes[2].text(0.2, 9.5, f'{Q2/total*100:.1f}%', fontsize=12, color='red')
    axes[2].text(cutoff + 0.2, 9.5, f'{Q3/total*100:.1f}%', fontsize=12)
    axes[2].text(8.5, 0.5, f'{Q4/total*100:.1f}%', fontsize=12, color='blue')

    axes[2].legend(loc='upper right')
    axes[2].grid(True, linestyle=':', alpha=0.5)
    plt.tight_layout()
    plt.show()

bias_slider = widgets.FloatSlider(
    value=0,
    min=-50,
    max=50,
    step=5,
    description='Biais systématique (%)',
    layout=widgets.Layout(width='400px'),
    style={'description_width': '180px'}
)

noise_slider = widgets.FloatSlider(
    value=0,
    min=0,
    max=0.8,
    step=0.01,
    description='Écart-type bruit',
    layout=widgets.Layout(width='400px'),
    style={'description_width': '180px'}
)

cutoff_slider = widgets.FloatSlider(
    value=2,
    min=2,
    max=8,
    step=0.1,
    description='Teneur de coupure (ppm)',
    layout=widgets.Layout(width='400px'),
    style={'description_width': '180px'}
)

interact(plot_real_vs_estimated_bias,
         bias_percent=bias_slider,
         noise_std=noise_slider,
         cutoff=cutoff_slider);


interactive(children=(FloatSlider(value=0.0, description='Biais systématique (%)', layout=Layout(width='400px'…